# *A currency converter using real time data from API with LLM*

#### Imports

In [1]:
# libraries
import requests
from langchain_openai import ChatOpenAI
from langchain_community.tools import tool
from langchain_core.messages import HumanMessage

In [2]:
#API Key:
from dotenv import load_dotenv
load_dotenv()

True

#### Creating the tool

In [3]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_rate(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    
    Args:
        base_currency: str - The source currency code (e.g., USD)
        target_currency: str - The target currency code (e.g., INR)
    Returns:
        float: The conversion rate from base to target currency
    """
    
    url = f'https://v6.exchangerate-api.com/v6/653db55b557f2e895e615492/pair/{base_currency}/{target_currency}'
    
    response = requests.get(url)
    return response.json()


@tool
def convert(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Given a currency conversion rate, this function calculates the target currency value from a given base currency value
    Args:
        base_currency_value: float - The amount to convert
        conversion_rate: float - The conversion rate to apply
    Returns:
        float: The converted amount in target currency
    """
    return base_currency_value * conversion_rate

In [4]:
get_conversion_rate.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1760745601,
 'time_last_update_utc': 'Sat, 18 Oct 2025 00:00:01 +0000',
 'time_next_update_unix': 1760832001,
 'time_next_update_utc': 'Sun, 19 Oct 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.0103}

In [5]:
convert.invoke({'base_currency_value':10,'conversion_rate':87.58})

875.8

#### Creating LLM obj

In [6]:
llm = ChatOpenAI()

#### Binding tool with llm

In [7]:
llm_with_tools = llm.bind_tools([get_conversion_rate, convert])

In [8]:
message = [HumanMessage("What is the conversion factor between INR and USD, and based on that can you convert 100000 inr to usd")]
message

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 100000 inr to usd', additional_kwargs={}, response_metadata={})]

In [9]:
ai_message = llm_with_tools.invoke(message)

In [10]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NZAIz2RO0fLMBfAV8QiPHkqA', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_rate'}, 'type': 'function'}, {'id': 'call_9lSkp0z3RzdGcM2ym3XAZvcV', 'function': {'arguments': '{"base_currency_value": 100000}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 217, 'total_tokens': 270, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CRsDtASwp2qCSfFUIUBxlX9pPccpt', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b4828281-84a4-41b5-9409-819f4ac058a0-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'base_cur

In [11]:
message.append(ai_message)

In [12]:
ai_message.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_NZAIz2RO0fLMBfAV8QiPHkqA',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 100000},
  'id': 'call_9lSkp0z3RzdGcM2ym3XAZvcV',
  'type': 'tool_call'}]

In [14]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_rate':
    tool_message1 = get_conversion_rate.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    message.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    message.append(tool_message2)

In [15]:
message

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 100000 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NZAIz2RO0fLMBfAV8QiPHkqA', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_rate'}, 'type': 'function'}, {'id': 'call_9lSkp0z3RzdGcM2ym3XAZvcV', 'function': {'arguments': '{"base_currency_value": 100000}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 217, 'total_tokens': 270, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CRsDtASwp2qCSfFUIUBxlX9pPccpt', 'service_tier':

In [16]:
llm_with_tools.invoke(message)

AIMessage(content='The conversion factor between INR and USD is 0.01136. \n\nSo, if you convert 100,000 INR to USD, the equivalent amount is 1136 USD.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 430, 'total_tokens': 470, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CRsEtI6wEya9YjnEv7gji3dTAc4z5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--af14ee4b-c785-4bd9-ba99-4b43ef3bd158-0', usage_metadata={'input_tokens': 430, 'output_tokens': 40, 'total_tokens': 470, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})